# Import Data

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Import data from excel
xls = pd.ExcelFile("hk_603ipo_mrk_value.xlsm")

In [ ]:
df = pd.read_excel(xls, 'Sheet1')
df['Name']=df['Name'].str.strip("<>")
df.columns = df.columns.astype(str)
df.columns = df.columns.str.replace(" 00:00:00","")
df

In [ ]:
# Import TR and Return1Year data
tr = pd.read_csv('data_TR.csv',usecols=['Instrument','IPO Date','TRBC Economic Sector Name'])
tr["IPO Date"] = pd.to_datetime(tr["IPO Date"]).dt.strftime('%Y-%m-%d')

stockReturn = pd.read_excel('Short_performance2.xlsx')[['Company','return_1year']]
stockReturn["Company"] = stockReturn["Company"].astype(str).str.cat(['.HK']*603)

tr = tr.merge(stockReturn, left_on='Instrument', right_on='Company')
tr

# Find peer groups and MktCap

In [ ]:
ipodate = tr.loc[tr["Instrument"] == "1353.HK", "IPO Date"].values[0]
marketcap = df.loc[df['Name'] == "1353.HK",ipodate].values[0]
print(str(ipodate) + " " + str(marketcap))

In [ ]:
# get list of stocks
stocknames = tr['Instrument'].values

# create dataframe with MktCap and return_1year
data = pd.DataFrame(columns = ['Stock','Industry','IPO','MktCap', "return_1year"])
for stock in stocknames:
    try:
        industry = tr.loc[tr["Instrument"] == stock, "TRBC Economic Sector Name"].values[0]
        return_1year = tr.loc[tr["Instrument"] == stock, "return_1year"].values[0]
        ipodate = tr.loc[tr["Instrument"] == stock, "IPO Date"].values[0]
        marketcap = df.loc[df["Name"] == stock, ipodate].values[0]
        if pd.isna(marketcap): continue
        data = data.append({'Stock': stock, 'Industry': industry, 'IPO': ipodate, 'MktCap': marketcap, 'return_1year': return_1year}, ignore_index=True)
    except KeyError as e:
        #print(str(stock) + " " + str(e))
        continue

data

In [ ]:
# Find company with best performance among peers
graph_MktCap = []
graph_Return1Year = []

for industry in data.Industry.unique():
    
    divisions = 10 if industry in ["Consumer Cyclicals", "Industrials"] else 1 if industry in ["Utilities", "Energy"] else 4
    size = int(len(data.loc[data["Industry"] == industry].sort_values(by=['MktCap']))/divisions)
    remainder = len(data.loc[data["Industry"] == industry].sort_values(by=['MktCap']))%divisions
    
    print("Industry: " + str(industry))
    mc, r1y = [], []
    
    for d in range(divisions):
        
        start = d*size
        end = (d+1)*size+1 if d<remainder else (d+1)*size
        peers = data.loc[data["Industry"] == industry].sort_values(by=['MktCap']).iloc[start:end]
        
        #print(peers.sort_values(by=['return_1year'], ascending=False).iloc[0:1])
        best = peers.sort_values(by=['return_1year'], ascending=False).iloc[0:1]
        stock = best["Stock"].values[0]
        MktCap = round(best["MktCap"].values[0], 3)
        Return1Year = round(best['return_1year'].values[0], 5)
        
        print("Division {} - [Stock: {}] [MktCap: {}] [Return1Year: {}]".format(d+1, stock, MktCap, Return1Year))
        mc.append(MktCap)
        r1y.append(Return1Year)
    
    graph_MktCap.append(mc)
    graph_Return1Year.append(r1y)
    print("=====END=====\n")

In [ ]:
# Find average performance of peer groups
graph_avgMktCap = []
graph_avgReturn1Year = []

for industry in data.Industry.unique():
    
    divisions = 10 if industry in ["Consumer Cyclicals", "Industrials"] else 1 if industry in ["Utilities", "Energy"] else 4
    size = int(len(data.loc[data["Industry"] == industry].sort_values(by=['MktCap']))/divisions)
    remainder = len(data.loc[data["Industry"] == industry].sort_values(by=['MktCap']))%divisions
    
    print("Industry: " + str(industry))
    mc, r1y = [], []
    
    for d in range(divisions):
    
        start = d*size
        end = (d+1)*size+1 if d<remainder else (d+1)*size
        peers = data.loc[data["Industry"] == industry].sort_values(by=['MktCap']).iloc[start:end]
        
        avgMktCap = round(peers["MktCap"].mean(), 3)
        avgReturn1Year = round(peers['return_1year'].mean(), 5)
        
        print("Division {} - [avgMktCap: {}] [avgReturn1Year: {}]".format(d+1, avgMktCap, avgReturn1Year))
        mc.append(avgMktCap)
        r1y.append(avgReturn1Year)
        
    graph_avgMktCap.append(mc)
    graph_avgReturn1Year.append(r1y)
    print("=====END=====\n")

# Plot against Return1Year

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot company with best performance among peers
x1 = graph_MktCap
y1 = graph_Return1Year

industries = data.Industry.unique()
for i in range(len(industries)):
    plt.figure(figsize=(4, 2))
    plt.title(industries[i])
    plt.plot(x1[i],y1[i])

In [ ]:
# Plot average performance of peer groups
x2 = graph_avgMktCap
y2 = graph_avgReturn1Year

industries = data.Industry.unique()
for i in range(len(industries)):
    plt.figure(figsize=(4, 2))
    plt.title(industries[i])
    plt.plot(x1[i],y1[i])